In [1]:
#匯入資料 
#安隆公司曾是一間能源公司，2001 年破產前是世界上最大的電力、天然氣及電信公司之一。
#擁有上千億資產的公司於 2002 年竟然在短短幾周內宣告破產，才揭露其財報在多年以來均是造假的醜聞。
#在本資料集中你將會扮演偵探的角色，透過高層經理人內部的 mail 來往的情報以及薪資、股票等財務特徵，
#訓練出一個機器學習模型來幫忙你找到可疑的詐欺犯罪者是誰! 
#我們已經先幫你找到幾位犯罪者 (Person-of-Interest, poi) 與清白的員工，請利用這些訓練資料來訓練屬於自己的詐欺犯機器學習模型吧!
import pandas as pd
data_path = 'data/'
train = pd.read_csv(data_path + 'train_data.csv' )  
test = pd.read_csv(data_path + 'test_features.csv' )
print(train.shape)
print(test.shape)

(113, 22)
(33, 21)


In [2]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 21 columns):
name                         33 non-null object
bonus                        21 non-null float64
deferral_payments            11 non-null float64
deferred_income              15 non-null float64
director_fees                4 non-null float64
email_address                28 non-null object
exercised_stock_options      21 non-null float64
expenses                     22 non-null float64
from_messages                21 non-null float64
from_poi_to_this_person      21 non-null float64
from_this_person_to_poi      21 non-null float64
loan_advances                2 non-null float64
long_term_incentive          17 non-null float64
other                        24 non-null float64
restricted_stock             28 non-null float64
restricted_stock_deferred    8 non-null float64
salary                       22 non-null float64
shared_receipt_with_poi      21 non-null float64
to_messages         

In [3]:
train.info()  #有missing DATA, 亦可以寫 df_train.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 22 columns):
name                         113 non-null object
bonus                        61 non-null float64
deferral_payments            28 non-null float64
deferred_income              34 non-null float64
director_fees                13 non-null float64
email_address                83 non-null object
exercised_stock_options      81 non-null float64
expenses                     73 non-null float64
from_messages                65 non-null float64
from_poi_to_this_person      65 non-null float64
from_this_person_to_poi      65 non-null float64
loan_advances                2 non-null float64
long_term_incentive          49 non-null float64
other                        69 non-null float64
poi                          113 non-null bool
restricted_stock             82 non-null float64
restricted_stock_deferred    10 non-null float64
salary                       73 non-null float64
shared_receipt_wi

In [4]:
train.head() 

,name,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,...,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,RICE KENNETH D,1750000.0,NaN,-3504386.0,NaN,ken.rice@enron.com,19794175.0,46950.0,18.0,42.0,...,1617011.0,174839.0,True,2748364.0,NaN,420636.0,864.0,905.0,505050.0,22542539.0
1,SKILLING JEFFREY K,5600000.0,NaN,NaN,NaN,jeff.skilling@enron.com,19250000.0,29336.0,108.0,88.0,...,1920000.0,22122.0,True,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,SHELBY REX,200000.0,NaN,-4167.0,NaN,rex.shelby@enron.com,1624396.0,22884.0,39.0,13.0,...,NaN,1573324.0,True,869220.0,NaN,211844.0,91.0,225.0,2003885.0,2493616.0
3,KOPPER MICHAEL J,800000.0,NaN,NaN,NaN,michael.kopper@enron.com,NaN,118134.0,NaN,NaN,...,602671.0,907502.0,True,985032.0,NaN,224305.0,NaN,NaN,2652612.0,985032.0
4,CALGER CHRISTOPHER F,1250000.0,NaN,-262500.0,NaN,christopher.calger@enron.com,NaN,35818.0,144.0,199.0,...,375304.0,486.0,True,126027.0,NaN,240189.0,2188.0,2598.0,1639297.0,126027.0


In [5]:
train.describe()

,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
count,6.100000e+01,2.800000e+01,3.400000e+01,13.000000,8.100000e+01,73.000000,65.000000,65.000000,65.000000,2.000000e+00,4.900000e+01,6.900000e+01,8.200000e+01,10.000000,7.300000e+01,65.000000,65.000000,9.600000e+01,9.800000e+01
mean,1.147436e+06,6.344374e+05,-4.625664e+05,89397.846154,2.985081e+06,51040.547945,711.323077,64.800000,40.092308,4.096250e+07,7.926171e+05,4.471774e+05,1.294855e+06,-221885.700000,2.739025e+05,1111.369231,2156.061538,2.590977e+06,3.527136e+06
std,1.505189e+06,8.603646e+05,8.095392e+05,41143.391399,6.004174e+06,47596.682104,2074.497628,91.863214,88.901407,5.736404e+07,9.504645e+05,1.341564e+06,2.498335e+06,205191.374121,1.716647e+05,1165.852016,2811.676718,1.056645e+07,7.182997e+06
min,7.000000e+04,-1.025000e+05,-3.504386e+06,3285.000000,3.285000e+03,148.000000,12.000000,0.000000,0.000000,4.000000e+05,7.102300e+04,2.000000e+00,4.409300e+04,-560222.000000,4.770000e+02,2.000000,57.000000,1.480000e+02,-4.409300e+04
25%,4.500000e+05,7.656750e+04,-5.527032e+05,101250.000000,4.004780e+05,18834.000000,19.000000,10.000000,0.000000,2.068125e+07,2.750000e+05,9.720000e+02,2.689220e+05,-389621.750000,2.061210e+05,178.000000,517.000000,3.024025e+05,4.211518e+05
50%,7.500000e+05,1.951900e+05,-1.175340e+05,108579.000000,8.500100e+05,41953.000000,45.000000,28.000000,7.000000,4.096250e+07,4.221580e+05,5.238200e+04,4.628225e+05,-139856.500000,2.516540e+05,599.000000,1088.000000,1.106740e+06,9.979710e+05
75%,1.000000e+06,8.342052e+05,-2.708325e+04,112492.000000,2.165172e+06,59175.000000,215.000000,88.000000,27.000000,6.124375e+07,8.318090e+05,3.620960e+05,9.664905e+05,-77953.250000,2.885890e+05,1902.000000,2649.000000,1.985668e+06,2.493616e+06
max,8.000000e+06,2.964506e+06,-1.042000e+03,125034.000000,3.434838e+07,228763.000000,14368.000000,528.000000,411.000000,8.152500e+07,5.145434e+06,1.035973e+07,1.476169e+07,44093.000000,1.111258e+06,4527.000000,15149.000000,1.035598e+08,4.911008e+07


In [6]:
#有關財務的特徵: ['salary', 'deferral_payments', 'total_payments', 'loan_advances', 'bonus', '
# restricted_stock_deferred', 'deferred_income', 'total_stock_value', 'expenses', 
# 'exercised_stock_options', 'other', 'long_term_incentive', 'restricted_stock', 'director_fees'] (單位皆為美元)。

#有關 email 的特徵: ['to_messages', 'email_address', 'from_poi_to_this_person', 
# 'from_messages', 'from_this_person_to_poi', 'shared_receipt_with_poi'] (除了 email_address，其餘皆為次數)

#嫌疑人的標記，也就是我們常用的 **y**。POI label: [‘poi’] (boolean, represented as integer)
train  = train[train['name']!='THE TRAVEL AGENCY IN THE PARK'] 
print(train.shape)
df_train_Y = train['poi'].astype(int)
print(df_train_Y)
ids = test['name']
df_train = train.drop(['name', 'poi'] , axis=1)
df_test = test.drop(['name'] , axis=1)


(112, 22)
0      1
1      1
2      1
3      1
4      1
5      1
6      1
7      1
8      1
9      1
10     1
11     1
12     1
13     0
14     0
15     0
16     0
17     0
18     0
19     0
20     0
21     0
22     0
23     0
24     0
25     0
26     0
27     0
28     0
29     0
      ..
83     0
84     0
85     0
86     0
87     0
88     0
89     0
90     0
91     0
92     0
93     0
94     0
95     0
96     0
97     0
98     0
99     0
100    0
101    0
102    0
103    0
104    0
105    0
106    0
107    0
108    0
109    0
110    0
111    0
112    0
Name: poi, Length: 112, dtype: int32


In [7]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df_train.dtypes, df_train.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

19 Numeric Features : ['bonus', 'deferral_payments', 'deferred_income', 'director_fees', 'exercised_stock_options', 'expenses', 'from_messages', 'from_poi_to_this_person', 'from_this_person_to_poi', 'loan_advances', 'long_term_incentive', 'other', 'restricted_stock', 'restricted_stock_deferred', 'salary', 'shared_receipt_with_poi', 'to_messages', 'total_payments', 'total_stock_value']



In [8]:
import matplotlib.pyplot as plt
# 檢視這些欄位的數值範圍
for col in num_features:
    df_train[[col]].boxplot()
    plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

...

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [9]:
# 削減文字型欄位, 只剩數值型欄位
df_train_num = df_train[num_features]

#補值
#參考 Day_019_Fill_NaN_and_Scalers 課程, 連續變數(ex.int64, float64) 使用 -1, 0, 平均值 補植
df_train_mn = df_train_num.fillna(0) #  0 補植
df_train_mn

,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,1750000.0,0.0,-3504386.0,0.0,19794175.0,46950.0,18.0,42.0,4.0,0.0,1617011.0,174839.0,2748364.0,0.0,420636.0,864.0,905.0,505050.0,22542539.0
1,5600000.0,0.0,0.0,0.0,19250000.0,29336.0,108.0,88.0,30.0,0.0,1920000.0,22122.0,6843672.0,0.0,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,200000.0,0.0,-4167.0,0.0,1624396.0,22884.0,39.0,13.0,14.0,0.0,0.0,1573324.0,869220.0,0.0,211844.0,91.0,225.0,2003885.0,2493616.0
3,800000.0,0.0,0.0,0.0,0.0,118134.0,0.0,0.0,0.0,0.0,602671.0,907502.0,985032.0,0.0,224305.0,0.0,0.0,2652612.0,985032.0
4,1250000.0,0.0,-262500.0,0.0,0.0,35818.0,144.0,199.0,25.0,0.0,375304.0,486.0,126027.0,0.0,240189.0,2188.0,2598.0,1639297.0,126027.0
5,0.0,10259.0,0.0,0.0,30766064.0,77978.0,0.0,0.0,0.0,0.0,0.0,2856.0,0.0,0.0,0.0,0.0,0.0,91093.0,30766064.0
6,0.0,0.0,0.0,0.0,8308552.0,53947.0,0.0,0.0,0.0,0.0,0.0,147950.0,3576206.0,0.0,158403.0,0.0,0.0,360300.0,11884758.0
7,1200000.0,27610.0,-144062.0,0.0,0.0,16514.0,40.0,240.0,11.0,0.0,0.0,101740.0,698242.0,0.0,288542.0,1132.0,1758.0,1490344.0,698242.0
8,7000000.0,202911.0,-300000.0,0.0,34348384.0,99832.0,36.0,123.0,16.0,81525000.0,3600000.0,10359729.0,14761694.0,0.0,1072321.0,2411.0,4273.0,103559793.0,49110078.0
9,600000.0,0.0,0.0,0.0,384728.0,125978.0,16.0,52.0,6.0,0.0,71023.0,200308.0,393818.0,0.0,274975.0,874.0,873.0,1272284.0,778546.0


In [10]:
 #只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df_train.dtypes, df_train.columns):
    if dtype == 'object' or dtype == 'bool':
        object_features.append(feature)
print(f'{len(object_features)} Object Features : {object_features}\n')

# 只留類別型欄位
df_train_obj = df_train[object_features]
df_train_obj = df_train.fillna('None')

1 Object Features : ['email_address']



In [11]:
# Day_022_LabelEncoder_and_OneHotEncoder
# Day_023_Mean_Encoder 
# Day_024_CountEncoder_and_FeatureHash 
#from sklearn.preprocessing import LabelEncoder
#df_train_Label = pd.DataFrame()
#for c in object_features:
    #df_train_Label[c] = LabelEncoder().fit_transform(df_train_Label[c])
    #df_train_Label[c] = pd.get_dummies(df_train_Label[c])
#df_train_Label
 
df_train_hash = pd.DataFrame()
for c in object_features:
    df_train_hash[c] = df_train_obj[c].map(lambda x:hash(x) % 10)
df_train_hash

,email_address
0,6
1,1
2,5
3,3
4,1
5,4
6,8
7,7
8,9
9,8


In [12]:
df_train_X = pd.concat([df_train_mn, df_train_hash], axis = 1)
df_train_X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112 entries, 0 to 112
Data columns (total 20 columns):
bonus                        112 non-null float64
deferral_payments            112 non-null float64
deferred_income              112 non-null float64
director_fees                112 non-null float64
exercised_stock_options      112 non-null float64
expenses                     112 non-null float64
from_messages                112 non-null float64
from_poi_to_this_person      112 non-null float64
from_this_person_to_poi      112 non-null float64
loan_advances                112 non-null float64
long_term_incentive          112 non-null float64
other                        112 non-null float64
restricted_stock             112 non-null float64
restricted_stock_deferred    112 non-null float64
salary                       112 non-null float64
shared_receipt_with_poi      112 non-null float64
to_messages                  112 non-null float64
total_payments               112 non-null flo

In [13]:
from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
# 切分訓練集/測試集
x, y = df_train_X, df_train_Y
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=4)

# 建立模型
clf = GradientBoostingClassifier()

# 預設模型準確率
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

acc = metrics.accuracy_score(y_test, y_pred)
print("Acuuracy: ", acc)

Acuuracy:  0.8214285714285714


In [14]:
# 設定要訓練的超參數組合
n_estimators = [50, 100, 150]
max_depth = [1, 3, 5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    4.2s finished


In [15]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: -0.095238 using {'max_depth': 1, 'n_estimators': 50}


In [16]:
# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingClassifier(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred2 = clf_bestparam.predict(x_test)

acc = metrics.accuracy_score(y_test, y_pred2)
print("Acuuracy: ", acc)

Acuuracy:  0.8928571428571429


In [17]:
#############################################
######  test ###############
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df_test.dtypes, df_test.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df_test_num = df_test[num_features]

#補值
df_test_mn = df_test_num.fillna(0) 
print(df_test_mn.shape)

19 Numeric Features : ['bonus', 'deferral_payments', 'deferred_income', 'director_fees', 'exercised_stock_options', 'expenses', 'from_messages', 'from_poi_to_this_person', 'from_this_person_to_poi', 'loan_advances', 'long_term_incentive', 'other', 'restricted_stock', 'restricted_stock_deferred', 'salary', 'shared_receipt_with_poi', 'to_messages', 'total_payments', 'total_stock_value']

(33, 19)


In [18]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df_test.dtypes, df_test.columns):
    if dtype == 'object' or dtype == 'bool':
        object_features.append(feature)
print(f'{len(object_features)} Object Features : {object_features}\n')

# 只留類別型欄位
df_test_obj = df_test[object_features]
df_test_obj = df_test.fillna('None')
 
# Day_024_CountEncoder_and_FeatureHash 
df_test_hash = pd.DataFrame()
for c in object_features:
    df_test_hash[c] = df_test_obj[c].map(lambda x:hash(x) % 10)
print(df_test_hash.shape)

1 Object Features : ['email_address']

(33, 1)


In [19]:
df_test_X = pd.concat([df_test_mn, df_test_hash], axis = 1)
df_test_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 20 columns):
bonus                        33 non-null float64
deferral_payments            33 non-null float64
deferred_income              33 non-null float64
director_fees                33 non-null float64
exercised_stock_options      33 non-null float64
expenses                     33 non-null float64
from_messages                33 non-null float64
from_poi_to_this_person      33 non-null float64
from_this_person_to_poi      33 non-null float64
loan_advances                33 non-null float64
long_term_incentive          33 non-null float64
other                        33 non-null float64
restricted_stock             33 non-null float64
restricted_stock_deferred    33 non-null float64
salary                       33 non-null float64
shared_receipt_with_poi      33 non-null float64
to_messages                  33 non-null float64
total_payments               33 non-null float64
total_stock_val

In [20]:
test_y = clf_bestparam.predict_proba(df_test_X)
test_y

array([[0.66242718, 0.33757282],
       [0.8906741 , 0.1093259 ],
       [0.7729653 , 0.2270347 ],
       [0.92667473, 0.07332527],
       [0.89990605, 0.10009395],
       [0.79274981, 0.20725019],
       [0.08791823, 0.91208177],
       [0.97030635, 0.02969365],
       [0.95680048, 0.04319952],
       [0.93880913, 0.06119087],
       [0.9709074 , 0.0290926 ],
       [0.97348387, 0.02651613],
       [0.97348387, 0.02651613],
       [0.97348387, 0.02651613],
       [0.51520372, 0.48479628],
       [0.97348387, 0.02651613],
       [0.95680048, 0.04319952],
       [0.92115548, 0.07884452],
       [0.82864277, 0.17135723],
       [0.97348387, 0.02651613],
       [0.09789186, 0.90210814],
       [0.79274981, 0.20725019],
       [0.96136573, 0.03863427],
       [0.94516671, 0.05483329],
       [0.96136573, 0.03863427],
       [0.97030635, 0.02969365],
       [0.94516671, 0.05483329],
       [0.96136573, 0.03863427],
       [0.87255246, 0.12744754],
       [0.84947558, 0.15052442],
       [0.

In [21]:
myDataFrame = pd.DataFrame(test_y[:,1],columns=['poi'],dtype='float64')
import numpy as np
index_column = np.arange(1,33)
myDataFrame['name'] = ids
myDataFrame

,poi,name
0,0.337573,BELDEN TIMOTHY N
1,0.109326,BOWEN JR RAYMOND M
2,0.227035,HANNON KEVIN P
3,0.073325,DELAINEY DAVID W
4,0.100094,CAUSEY RICHARD A
5,0.207250,HICKERSON GARY J
6,0.912082,FREVERT MARK A
7,0.029694,CHAN RONNIE
8,0.043200,DONAHUE JR JEFFREY M
9,0.061191,REYNOLDS LAWRENCE


In [22]:
myDataFrame.to_csv('Submission_022003.csv', index=False)